In [1]:
import numpy as np
import pandas as pd
import pyaudio
import wave
import librosa
from scipy.io.wavfile import write
from keras.models import model_from_json
from sklearn.preprocessing import StandardScaler

In [189]:
df = pd.read_csv('data/final_data.csv',index_col=0)

In [190]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,emotion
0,-587.907898,36.178154,-17.241106,-9.742208,-27.773447,-14.733717,-11.477688,-15.569857,-14.273412,-8.294398,...,3.212901,-1.833511,-2.232787,-4.604078,0.343586,-1.242408,-0.375359,3.929535,10.683189,sad
1,-509.770386,43.354965,-24.751860,8.810980,-8.211090,-24.347290,-9.572964,-15.536777,-10.063596,4.296363,...,1.836776,-8.388222,-7.970921,-3.082049,3.787357,-0.004449,2.402939,7.500911,7.930544,happy
2,-608.325256,91.134865,-8.422993,24.703096,1.783348,-23.475849,-8.582272,-3.728264,-11.954734,0.899885,...,5.030650,0.232143,2.143103,-1.058072,-1.230369,-0.428278,6.082251,10.974966,18.634037,calm
3,-504.740112,55.126217,-20.908773,13.133360,-6.350088,-7.488061,-18.778595,-4.136145,-18.639141,-2.384829,...,2.723440,2.149348,0.681944,-4.007479,-1.396597,-1.036337,5.015131,10.885223,16.854820,happy
4,-637.394531,43.984432,-17.269554,-3.644861,-15.109137,-18.893116,-2.212192,-20.164730,-10.050422,2.773326,...,15.225397,0.327097,-4.202260,-5.842893,1.823740,-2.011576,3.554226,11.488029,20.341881,calm


In [191]:
# Define constants for audio recording
format = pyaudio.paInt16
channels = 1
filename = "test/output.wav"
duration=5
sample_rate=44100
chunk_size=1024
duration=5

# Function to record audio
def record_audio():
    audio = pyaudio.PyAudio()
    stream = audio.open(format=format, channels=channels,
                        rate=sample_rate, input=True,
                        frames_per_buffer=chunk_size)
    print("Recording...")
    frames = []

    # Calculate total number of chunks
    total_chunks = int(sample_rate / chunk_size * duration)
    chunks_per_marker = total_chunks // 10  # Number of chunks for each '#' character
    
    for i in range(total_chunks):
        data = stream.read(chunk_size)
        frames.append(data)
        percentage = int((i + 1) / total_chunks * 100)
        if (i + 1) % chunks_per_marker == 0:
            print(f" -- {percentage}%", end='', flush=True)  # Print '#' without newline and flush buffer
    
    print("\nFinished recording.")
    
    stream.stop_stream()
    stream.close()
    audio.terminate()
    
    wavefile = wave.open(filename, 'wb')
    wavefile.setnchannels(channels)
    wavefile.setsampwidth(audio.get_sample_size(format))
    wavefile.setframerate(sample_rate)
    wavefile.writeframes(b''.join(frames))
    wavefile.close()

In [192]:
record_audio()

Recording...
 -- 9% -- 19% -- 29% -- 39% -- 48% -- 58% -- 68% -- 78% -- 87% -- 97%
Finished recording.


In [193]:
#sample feature
#librosa.core.load(path, sr=22050, mono=True, offset=0.0, duration=None, dtype=<class 'numpy.float32'>, res_type='kaiser_best')
res_type_s = 'kaiser_best'
duration_s = None
sample_rate_s = 22050
offset_s = 0.5

#Mfcc
#librosa.feature.mfcc(y=None, sr=22050, S=None, n_mfcc=20, dct_type=2, norm='ortho', lifter=0, **kwargs)
mfcc_sample_rate = 22050
n_mfcc = 40
axis_mfcc = 1

audio_file = pd.DataFrame(columns=['feature'])
filename='test/output.wav'
X, sample_rate = librosa.load(filename, 
                              res_type = res_type_s,
                              duration = duration_s,
                              sr = sample_rate_s,
                              offset = offset_s)
sample_rate = np.array(sample_rate)

#Get MFCCs from each file
mfccs = librosa.feature.mfcc(y=X, sr = mfcc_sample_rate,  n_mfcc = n_mfcc)

#Calculate mean of MFCCs
mfccs_mean = np.mean(    mfccs, 
                         axis = axis_mfcc)
feature = mfccs_mean

#Add MFCCs feature results to list
audio_file.loc[0] = [feature]

In [194]:
audio_file

,feature
0,"[-333.60132, 91.907295, 6.3223624, 3.3208039, ..."


In [195]:
X = df.iloc[:,:-1]

In [196]:
X

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,-587.907898,36.178154,-17.241106,-9.742208,-27.773447,-14.733717,-11.477688,-15.569857,-14.273412,-8.294398,...,-2.337345,3.212901,-1.833511,-2.232787,-4.604078,0.343586,-1.242408,-0.375359,3.929535,10.683189
1,-509.770386,43.354965,-24.751860,8.810980,-8.211090,-24.347290,-9.572964,-15.536777,-10.063596,4.296363,...,1.311170,1.836776,-8.388222,-7.970921,-3.082049,3.787357,-0.004449,2.402939,7.500911,7.930544
2,-608.325256,91.134865,-8.422993,24.703096,1.783348,-23.475849,-8.582272,-3.728264,-11.954734,0.899885,...,0.012662,5.030650,0.232143,2.143103,-1.058072,-1.230369,-0.428278,6.082251,10.974966,18.634037
3,-504.740112,55.126217,-20.908773,13.133360,-6.350088,-7.488061,-18.778595,-4.136145,-18.639141,-2.384829,...,0.796294,2.723440,2.149348,0.681944,-4.007479,-1.396597,-1.036337,5.015131,10.885223,16.854820
4,-637.394531,43.984432,-17.269554,-3.644861,-15.109137,-18.893116,-2.212192,-20.164730,-10.050422,2.773326,...,1.124650,15.225397,0.327097,-4.202260,-5.842893,1.823740,-2.011576,3.554226,11.488029,20.341881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5247,-433.188965,57.363720,4.908972,13.024331,-3.378766,-0.960908,-15.769565,4.778440,-14.686361,-8.497721,...,1.629190,6.618638,6.445676,-1.611167,-1.723101,1.382978,4.430180,5.747354,-2.161623,3.657368
5248,-559.998840,101.993713,33.605961,12.951871,21.204279,13.413920,4.372688,-7.319698,-9.703876,1.709915,...,5.289425,16.355019,8.957122,11.558165,4.750055,4.040524,2.418961,6.325723,7.394610,15.063540
5249,-496.139191,133.011856,45.438461,0.555025,10.132165,11.738538,-6.431499,7.704493,-19.552839,-16.369890,...,-1.042813,0.150162,3.208148,6.247221,12.822376,13.872787,10.784860,12.104943,9.085450,10.249683
5250,-353.824036,98.714981,-1.281175,8.937387,-3.878722,-3.579316,-3.132740,-20.462267,-5.338827,8.199279,...,-12.223659,-2.974227,-11.691386,-2.699654,3.988725,22.577427,33.460434,37.656155,27.965282,13.931880


In [197]:
new_df = pd.DataFrame(audio_file['feature'].values.tolist())

In [198]:
new_df

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,-333.601318,91.907295,6.322362,3.320804,2.049856,-1.398524,-4.375069,-0.899,-7.297161,-6.90255,...,-4.849685,-2.901773,-1.001762,-0.132825,-0.82394,-1.280829,-3.100461,-1.861721,0.314752,0.035467


In [199]:
X.columns

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36',
       '37', '38', '39'],
      dtype='object')

In [200]:
new_df.columns = X.columns

In [201]:
new_df.columns

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36',
       '37', '38', '39'],
      dtype='object')

In [202]:
frames = [X,new_df]
X = pd.concat(frames, ignore_index=True)

In [203]:
X

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,-587.907898,36.178154,-17.241106,-9.742208,-27.773447,-14.733717,-11.477688,-15.569857,-14.273412,-8.294398,...,-2.337345,3.212901,-1.833511,-2.232787,-4.604078,0.343586,-1.242408,-0.375359,3.929535,10.683189
1,-509.770386,43.354965,-24.751860,8.810980,-8.211090,-24.347290,-9.572964,-15.536777,-10.063596,4.296363,...,1.311170,1.836776,-8.388222,-7.970921,-3.082049,3.787357,-0.004449,2.402939,7.500911,7.930544
2,-608.325256,91.134865,-8.422993,24.703096,1.783348,-23.475849,-8.582272,-3.728264,-11.954734,0.899885,...,0.012662,5.030650,0.232143,2.143103,-1.058072,-1.230369,-0.428278,6.082251,10.974966,18.634037
3,-504.740112,55.126217,-20.908773,13.133360,-6.350088,-7.488061,-18.778595,-4.136145,-18.639141,-2.384829,...,0.796294,2.723440,2.149348,0.681944,-4.007479,-1.396597,-1.036337,5.015131,10.885223,16.854820
4,-637.394531,43.984432,-17.269554,-3.644861,-15.109137,-18.893116,-2.212192,-20.164730,-10.050422,2.773326,...,1.124650,15.225397,0.327097,-4.202260,-5.842893,1.823740,-2.011576,3.554226,11.488029,20.341881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5248,-559.998840,101.993713,33.605961,12.951871,21.204279,13.413920,4.372688,-7.319698,-9.703876,1.709915,...,5.289425,16.355019,8.957122,11.558165,4.750055,4.040524,2.418961,6.325723,7.394610,15.063540
5249,-496.139191,133.011856,45.438461,0.555025,10.132165,11.738538,-6.431499,7.704493,-19.552839,-16.369890,...,-1.042813,0.150162,3.208148,6.247221,12.822376,13.872787,10.784860,12.104943,9.085450,10.249683
5250,-353.824036,98.714981,-1.281175,8.937387,-3.878722,-3.579316,-3.132740,-20.462267,-5.338827,8.199279,...,-12.223659,-2.974227,-11.691386,-2.699654,3.988725,22.577427,33.460434,37.656155,27.965282,13.931880
5251,-495.061829,94.790359,25.488504,-16.077322,-3.615371,6.533344,-5.271453,0.514300,-17.379936,1.059698,...,1.496013,6.168132,-3.996353,-4.627251,0.300278,3.364248,-3.688179,-0.618601,-10.424759,0.341845


In [204]:
from sklearn.preprocessing import StandardScaler
#Normalize the data
scaler = StandardScaler()
X_scalled = scaler.fit_transform(X)

In [205]:
x_cnn = np.expand_dims(X_scalled, axis=2)

In [206]:
x_cnn.shape

(5253, 40, 1)

In [207]:
# Load the saved model
def load_model(model_json_path, model_weights_path):
    json_file = open(model_json_path, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights(model_weights_path)
    print("Loaded model from disk")
    return loaded_model

In [208]:
# Define paths
model_json_path = 'models/new_model.json'
model_weights_path = 'models/new_model.h5'

In [209]:
model = load_model(model_json_path,model_weights_path)

Loaded model from disk


In [210]:
y_test_predictions = np.argmax(model.predict(x_cnn), axis=-1)
y_test_predictions

array([6, 4, 1, ..., 5, 3, 2], dtype=int64)

In [211]:
emotions = {0: 'angry',
 1: 'calm',
 2: 'disgust',
 3: 'fear',
 4: 'happy',
 5: 'neutral',
 6: 'sad',
 7: 'surprised'}

In [212]:
y_test_predictions_labels =[]

#Go through each prediction and append to new list
for e in range(len(y_test_predictions)):
    y_test_predictions_labels.append(emotions[y_test_predictions[e]])
    
#Build array of predictions
y_test_predictions_labels = np.array(y_test_predictions_labels)
print(y_test_predictions_labels[-1].title())

Disgust
